In [1]:
import gpytorch
import time
import torch
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import FloatSlider, interact
from scipy.optimize import minimize
from types import SimpleNamespace as Namespace
from scipy.stats import multivariate_normal

In [2]:
import gp
import utils

In [3]:
%config InlineBackend.figure_format='retina'
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Active inference walking objective

We model the number of step for each interval $\Delta t$ (e.g. 5 min) with a Gaussian process that takes as input the context $c$. We then can compute estimate the number of total time step in the day, mean + SD. We can express the preference as a Gaussian distribution centered around the goal. 

Problem: it would be penalized if we expect more.

Find $Q^*(\pi)$ s.t. the Expected Free Energy, $\mathcal{G} = - \mathbb E_{Q ( o_{t:T}, z_{t:T}, \pi )}\left[ \ln Q(z_{t:T} \mid \pi ) - \ln \tilde p(o_{t:T}, z_{t: T}) \right]$, is minimized, and, interestingly:

$Q(\pi) = \sum_{t}^{T} \mathbb E_{Q(o_t, z_t \mid \pi)} \left[\ln Q(z_t \mid \pi) - \ln \tilde p(o_t, z_t) \right] = \sum_{t}^{T}  \underbrace{ \mathbb E_{Q(o_t, z_t \mid \pi)} D_{KL}\left[ Q(z_t \mid o_t, \pi) || Q(z_t \mid \pi) \right] }_{ \text{epistemic value} } + \underbrace{ \mathbb E_{Q(o_t \mid \pi) } \left[ \ln \tilde p(o_t) \right]}_{ \text{pragmatic value} } $